In [36]:
from bs4 import BeautifulSoup
import requests
import json

menu_url_nonDup = json.load(open('../recipe-clawler/menu_url_nonDup.json', 'r'))
menu_url_nonDup_doublet = list(menu_url_nonDup.items())

In [42]:
# Try one
page = requests.get(menu_url_nonDup_doublet[4][1]).content
soup = BeautifulSoup(page, "html.parser")

menu_url_nonDup_doublet[4]

('Chocolate Cake',
 'https://veenaazmanov.com/chocolate-cake-with-whipped-cream/')

In [78]:
from collections import defaultdict

menu_imgs = defaultdict(lambda: [])
img_not_detected = {}
content_err = {}

In [79]:
import sys
import time

for menu, url in menu_url_nonDup_doublet:

    if menu in menu_imgs: continue

    try:
        page = requests.get(url).content
    except Exception as e:
        print(f'Failed get: {menu}, {url}')
        continue
    
    soup = BeautifulSoup(page, "html.parser")

    content = soup.find_all('main', class_="content")
    try:
        assert len(content) == 1
    except AssertionError as e:
        content_err[menu] = url
        continue

    content = content[0]

    imgs = content.find_all('figure', class_="wp-block-image size-full")
    if len(imgs) == 0:
        imgs = content.find_all('figure', class_="wp-block-image")
        if len(imgs) == 0:
            img_not_detected[menu] = url
            continue

    for img in imgs:
        img_tag = img.img
        if not img_tag['alt'].lower().startswith("ingredients"):
            try:
                menu_imgs[menu].append(img_tag['data-lazy-src'])
            except KeyError as e:
                menu_imgs[menu].append(img_tag['src'])
            except KeyError as e:
                raise KeyError

    sys.stdout.write("\rData revieved: %i" % len(menu_imgs))
    sys.stdout.flush()
    time.sleep(0.1)

Data revieved: 363

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Data revieved: 742Failed get: Cancel reply, /recipes-in-text/#respond
Data revieved: 744

# Results

In [83]:
print(len(menu_imgs))
json.dump(menu_imgs, open("menu_imgs_url.json","w"))

744


In [80]:
content_err

{'All jam recipes': 'https://veenaazmanov.com/category/condiments-recipes/jam-recipes/'}

In [84]:
json.dump(img_not_detected, open("img_not_detected.json","w"))
img_not_detected

{'All buttercream Recipes here': 'https://veenaazmanov.com/category/cake-decorating-recipes/frosting-recipes/',
 'Eggless Strawberry Cupcakes': 'https://veenaazmanov.com/eggless-strawberry-cupcakes/',
 'cupcake recipes': 'https://veenaazmanov.com/category/cake-recipes/cupcake-recipes/',
 'Stenciled Christmas Sugar Cookies': 'https://veenaazmanov.com/stenciled-christmas-sugar-cookies/',
 'See all soup recipes': 'https://veenaazmanov.com/category/cooking-recipes/soup-recipes/',
 'See all salad recipes': 'https://veenaazmanov.com/category/cooking-recipes/salad-recipes/',
 'Chicken and Broccoli Zucchini Noodle Stir-Fry': 'https://veenaazmanov.com/chicken-and-broccoli-zucchini-noodle-stir-fry/',
 'All Bread Recipes': 'https://veenaazmanov.com/category/bread-recipes/',
 'NO churn Ice cream collection': 'https://veenaazmanov.com/category/cooking-recipes/ice-cream-popsicles-recipes/',
 'Indian Lychee Kulfi - Lychee Ice cream pops': 'https://veenaazmanov.com/homemade-lychee-kulfi/',
 'smoothies

> ## figure->div->img< alt have 'Progress', 'Ingredients'> : Not a dish's image